# 4-2. 컬러 스페이스
 - 영상에 색상과 명암을 표현하는 방법들과 각각의 차이
 - 각 방법의 활용 방법

#### 4-2-1. 디지털 영상의 종류
 - 디지털화된 이미지는 픽셀(화소)이라는 단위가 여러 개 모여서 그림을 표현함.
 - 하나의 픽셀을 어떻게 구성하느냐에 따라 이미지를 구분할 수 있음.

 - 1)바이너리(binary, 이진) 이미지
   * 한 개의 픽셀을 두 가지 값으로만 표현한 이미지. 통상 흑/백 이미지를 의미함.
   * 0과 1 또는 0과 255로 색상을 표현. 0은 검은색, 1 또는 255는 흰색.
   * 값으로는 명암을 표현할 수 없어 점의 밀도로 명암을 표현함(ex. 과거의 신문)

 - 2)그레이 스케일 이미지
   * 흔히 말하는 '흑백사진.'
   * 한 개의 픽셀을 0~255의 값으로 표현. 0에 가까울수록 어두운 검은색을, 255에 가까울수록 밝은 흰 색을 나타냄.
   * 음수가 없어서 1바이트의 크기로 표현하는 것이 일반적. 연산의 양을 줄여야 할 때 그레이 스케일 이미지를 사용.

 - 3)컬러 이미지
   * 한 픽셀당 0~255의 값 3개를 조합해서 표현함.
   * 컬러 스페이스(color space, 색공간): 각 바이트마다 어떤 색상 표현의 맡을지 결정하는 시스템
   * 컬러 스페이스의 종류: RGB, HSV, YUV(YCbCr), CMYK 등

#### 4-2-2. RGB, BGR, RGBA
 - RGB: Red, Green, Blue(빛의 3원소)
 - RGB이미지의 표현: 3차원 배열
    
                 행(row, height) x 열(column, height) x 채널(channel)

 - 행, 열: 영상 크기를 나타내는 지표
 - 각 색상은 0~255 범위로 표현하고, 값이 커질수록 해당 색상의 빛이 밝아짐(흰색: 색상값이 모두 255, 검은색: 모든 색상값이 0)
 - 채널: 세 가지 색상을 표현하는 차원

 - BRG: Blue, Green, Red
 - OpenCV는 위와 BGR 순서를 사용함!

 - RGBA: 배경의 투명 처리를 위해 alpha 채널을 추가한 것. 4번째 채널의 값을 0~255로 표현.

In [10]:
#practice. BGR, BGRA, Alpha 채널
#배경이 투명한 OpenCV 로고 이미지를 두 가지 옵션을 지정해서 비교.
import cv2
import numpy as np

img = cv2.imread('../img/opencv_logo.png')                    #기본값 옵션
bgr = cv2.imread('../img/opencv_logo.png', cv2.IMREAD_COLOR)  #IMREAD_COLOR 옵션
bgra = cv2.imread('../img/opencv_logo.png', cv2.IMREAD_UNCHANGED) #IMREAD_UNCHANGED 옵션

#각 옵션에 따른 이미지 shape
print("default: ", img.shape, "color: ", bgr.shape, "unchanged: ", bgra.shape)

cv2.imshow('bgr', bgr)
cv2.imshow('bgra', bgra)
cv2.imshow('alpha', bgra[:, :, 3]) #알파 채널만 표시
cv2.waitKey(0)
cv2.destroyAllWindows()

#img와 bgr 두 옵션은 shape가 동일함. 배경도 검은색이니 로고 아래 글씨가 안 보임.
#bgra는 채널이 하나 더 있음(, , 4)
#15번 라인에서 알파 채널만 떼어내서 따로 표시 => 로고, 글씨 외엔느 모두 검은색으로 표시
#즉 전경은 255, 배경은 0의 값을 줌으로써 전경과 배경을 손쉽게 분리할 수 있음 ==> 마스크 채널(mask channel)

default:  (120, 98, 3) color:  (120, 98, 3) unchanged:  (120, 98, 4)


In [12]:
print(bgra)

[[[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 ...

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]]


#### 4-2-3. 컬러 스페이스 변환
 - 변환의 목적(컬러 이미지 -> 그레이 스케일 변환): 이미지 연산의 양을 줄여 속도 향상
 - 애초에 그레이 스케일로 읽어올 수도 있지만, 처음에는 컬러 스케일로 불러들였다가 다른 컬러 스페이스로 변환해야 할 때도 많음.
 - 변환하는 방법
  * 1)변환 알고리즘 직접 구현 
  * 2)cv2.cvtColor()함수 이용.

 - out = cv2.cvtColor(img, flag)
  * img: Numpy 배열, 변환할 이미지.
  * flag: 변환할 컬러 스페이스. cv2.COLOR_로 시작하는 이름(274개)
    * cv2.COLOR_BGR2GRAY: BGR 컬러 이미지 -> 그레이 스케일
    * cv2.COLOR_GRAY2BGR: 그레이 스케일   -> BGR 컬러 이미지
        
             참고로, 실제로 흑백 사진을 컬러 사진으로 바꿔주는 것이 아님.
             2차원 배열 이미지를 3개 채널이 모두 같은 값을 갖는 3차원 배열로 변환하는 것임.
             영상 간에 연산을 할 때 차원이 다르면 연산이 불가능하므로 차원 맞추는 용도로 주로 사용.

    * cv2.COLOR_BGR2RGB:  BGR 컬러 이미지 -> RGB 컬러 이미지
    * cv2.COLOR_BGR2HSV:  BGR 컬러 이미지 -> HSV 컬러 이미지
    * cv2.COLOR_HSV2BGR:  HSV 컬러 이미지 -> BGR 컬러 이미지
    * cv2.COLOR_BGR2YUV:  BGR 컬러 이미지 -> YUV 컬러 이미지
    * cv2.COLOR_YUV2BGR:  YUV 컬러 이미지 -> BGR 컬러 이미지
  
  * out: 변환한 결과 이미지(Numpy 배열)

In [16]:
#practice. BGR을 그레이 스케일로 변환
import cv2
import numpy as np

img = cv2.imread('../img/girl.jpg')   #원본 이미지

#①~③: 평균값을 구하는 알고리즘 직접 구현
img2 = img.astype(np.uint16)         #dtype 변경 -- ①
                                     #(원래의 dtype이 uint8인 경우 평균값을 구하는 과정에서 3채널의 값을 합하면 255보다 큰 값이 나올 수 있음.)
b, g, r = cv2.split(img2)            #채널별로 분리 -- ②   
                                     #cv2.split(): 매개변수로 전달한 이미지를 채널별로 분리해서 튜플로 반환.
                                     # b, g, r = img2[:, :, 0], img2[:, :, 1], img2[:, :, 2] 와 동일한 코드!
gray1 = ((b+g+r)/3).astype(np.uint8) #평균값 연산 후 dtype 변경 -- ③   
                                     #uint16으로 계산을 먼저 마치고 다시 uint8로 변경.

#④: OpenCV 제공 함수를 통한 변환
gray2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  #BGR을 그레이스케일로 변경 -- ④

cv2.imshow('original', img)
cv2.imshow('gray1', gray1)
cv2.imshow('gray2', gray2)

cv2.waitKey(0)
cv2.destroyAllWindows()

#### 4-2-4. HSV, HSI, HSL
 - HSV: 3채널로 컬러 이미지 표시.
 - H(Hue, 색조), S(Saturation, 채도), V(Value, 명도)
 - I(Indensity, 밀도), L(Lightness, 명도())이나 여기서는 모두 같은 것으로 간주하고 HSV를 기준으로 정리.
  * H: 픽셀이 어떤 색인지를 표현. 빨강(165~180, 0~15), 초록(45~75), 파랑(90~120)
  * S: 채도 or 포화도 or 순도. 해당 색상이 얼마나 순수하게 포함되어있는가를 나타냄. 0~255의 범위를 지님(255가 가장 순수한 색상 표현).
  * V: 명도. 빛이 얼마나 밝은지, 어두운지를 표현함. 0~255의 범위를 가지며, 255가 가장 밝고 0이 가장 어두움(검은색).
  * BGR -> HSV 포맷 변환: cv2.cvtColor_BGR2HSV 또는 HSV2BGR

In [18]:
#practice. BGR에서 HSV로 변환
#완전한 빨강, 초록, 파랑 그리고 노랑을 BGR 포맷으로 표현해서 HSV로 변환하여 어떤 값인지 확인.
import numpy as np
import cv2

#BGR 컬러 스페이스로 원색 픽셀 생성 -- ①
red_bgr = np.array([[[0, 0, 255]]], dtype = np.uint8)   #빨강 값만 갖는 픽셀 
green_bgr = np.array([[[0, 255, 0]]], dtype = np.uint8) #초록 값만 갖는 픽셀
blue_bgr = np.array([[[255, 0, 0]]], dtype = np.uint8)   #파랑 값만 갖는 픽셀
yellow_bgr = np.array([[[0, 255, 255]]], dtype = np.uint8) #노랑 값만 ㄱ자는 픽셀

#BGR -> HSV 컬러 스페이스 변환
red_hsv = cv2.cvtColor(red_bgr, cv2.COLOR_BGR2HSV)
green_hsv = cv2.cvtColor(green_bgr, cv2.COLOR_BGR2HSV)
blue_hsv = cv2.cvtColor(blue_bgr, cv2.COLOR_BGR2HSV)
yellow_hsv = cv2.cvtColor(yellow_bgr, cv2.COLOR_BGR2HSV)

#HSV로 변환한 픽셀값 출력
print("red: ", red_hsv)
print("green: ", green_hsv)
print("blue: ", blue_hsv)
print("yellow: ", yellow_hsv)

red:  [[[  0 255 255]]]
green:  [[[ 60 255 255]]]
blue:  [[[120 255 255]]]
yellow:  [[[ 30 255 255]]]


#### 4-2-5. YUV, YCbCr
 - YUV 포맷: 색상을 인식할 때 밝기에 더 민감하고, 색상은 상대적으로 둔감한 점을 고려한 컬러 스페이스.
 - Y: 밝기(Luma)
 - U: 밝기와 파란색과의 색상 차(Chroma Blue, Cb)
 - V: 밝기와 빨간색과의 색상 차(Chroma Red, Cr)
 - Y에는 많은 비트수를, U와 V에는 적은 비트수를 할당해서 데이터를 압축하는 효과를 가짐.
 - 밝기 정보와 컬러 정보를 분리해서 사용하므로, 명암 대비(contrast)가 좋지 않은 영상을 좋게 만드는 데 활용됨.
 - YUV와 YCbCr의 차이
  * YUV: 텔레비전 시스템에서 아날로그 컬러 정보를 인코딩하는 데 사용함.
  * YCbCr포맷: MPEG, JPEG 등 디지털 컬러 정보를 인코딩하는 데 사용함.

In [20]:
#practice. BGR에서 YUV로 변환
import numpy as np
import cv2

#BGR 컬러 스페이스로 세 가지 밝기의 픽셀 생성
dark = np.array([[[0, 0, 0]]], dtype = np.uint8)         #가장 어두운 픽셀
middle = np.array([[[127, 127, 127]]], dtype = np.uint8) #중간 밝기 픽셀
bright = np.array([[[255, 255, 255]]], dtype = np.uint8) #가장 밝은 픽셀

#BGR -> YUV 컬러 스페이스 변환
dark_yuv = cv2.cvtColor(dark, cv2.COLOR_BGR2YUV)
middle_yuv = cv2.cvtColor(middle, cv2.COLOR_BGR2YUV)
bright_yuv = cv2.cvtColor(bright, cv2.COLOR_BGR2YUV)

#YUV로 변환한 픽셀 출력
print('dark_yuv', dark_yuv)
print('middle_yuv', middle_yuv)
print('bright_yuv', bright_yuv)
#YUV에서 밝기 정보는 첫 번째 열인 Y채널에서만 나타난다. 
#BGR은 3채널을 모두 연산해야 하지만 YUV는 Y 하나만 작업하면 되므로 효과적임.

dark_yuv [[[  0 128 128]]]
middle_yuv [[[127 128 128]]]
bright_yuv [[[255 128 128]]]
